# Dehergne repertoire analysis

> First time use: follow instructions in the [README.md](README.md) file in this directory.

## Setup

Create a TimelinkNotebook object. 

Note:
* First time run takes a little time as the required Docker images are downloaded.
* Timelink will default to using sqlite as the database, see [Receipts notebook for more control](1-receipts.ipynb)


In [1]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook()
tlnb.print_info(show_token=True)

Timelink version: 1.1.26
Project name: dehergne
Project home: /Users/jrc/mhk-home/sources/dehergne
Database type: sqlite
Database name: dehergne
Kleio image: timelinkserver/kleio-server
Kleio server token: NjFD1glAX5Gt1SBxMsfIysYXZD0jeO1d
Kleio server URL: http://127.0.0.1:8090
Kleio server home: /Users/jrc/mhk-home/sources/dehergne
Kleio server container: clever_lamport
Kleio version requested: latest
Kleio server version: 12.8.593 (2025-03-16 21:55:53)
SQLite directory: /Users/jrc/mhk-home/sources/dehergne/database/sqlite
Database version: 6ccf1ef385a6
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=dehergne, project_home=/Users/jrc/mhk-home/sources/dehergne, db_type=sqlite, db_name=dehergne, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=postgres, postgres_version=latest)


### Database status

Count the number of rows in each table in the database.


In [2]:
tlnb.table_row_count_df()

,table,count
0,acts,29
1,alembic_version,1
2,aregisters,1
3,attributes,26787
4,blinks,200
5,class_attributes,70
6,classes,14
7,entities,32995
8,geoentities,359
9,goods,0


### Check the status of the files

Check the import status of the translated files

* I: Imported
* E: Imported with error
* W: Imported with warnings no errors
* N: Not imported
* U: Translation updated need to reimport

In [3]:
kleio_files = tlnb.get_kleio_files()
# kleio_files.info()
kleio_files[["name","import_status","status","errors","warnings","import_errors","import_warnings"]]

,name,import_status,status,errors,warnings,import_errors,import_warnings
0,dehergne-0-abrev.cli,I,V,0,0,0,0
1,dehergne-a.cli,I,V,0,0,0,0
2,dehergne-b.cli,I,V,0,0,0,0
3,dehergne-c.cli,I,V,0,0,0,0
4,dehergne-d.cli,I,V,0,0,0,0
5,dehergne-e.cli,I,V,0,0,0,0
6,dehergne-f.cli,I,V,0,0,0,0
7,dehergne-g.cli,I,V,0,0,0,0
8,dehergne-h.cli,I,V,0,0,0,0
9,dehergne-i.cli,I,V,0,0,0,0


Can filter by places of entry using
the ids saved above in `place_of_entry_ids`

Optionally filter by place of entry: get list of ids of people from place of entry to be used as filter

In [26]:
place_of_stay=None# set to None to show all places

In [27]:
import pandas as pd
from timelink.pandas import entities_with_attribute
from dehergne_util import calc_age_at
# show 500 rows
pd.set_option('display.max_rows', 1550)

place_of_entry = "Coimbra"

jesuits_of_place = entities_with_attribute(
    entity_type='person',
    show_elements=['name','groupname','extra_info'],
    the_type='jesuita-entrada',
    the_value=place_of_entry,
    more_attributes=['dehergne','nascimento','embarque','morte'],
    db=tlnb.db,
)
place_of_entry_ids = jesuits_of_place.index.tolist()


In [28]:
import pandas as pd
from timelink.pandas import entities_with_attribute
# show 500 rows
pd.set_option('display.max_rows', 1550)

place_of_entry = "Coimbra"

ids = place_of_entry_ids

places_of_stay = entities_with_attribute(
    entity_type='person',
    show_elements=['name','groupname','the_line','the_order','extra_info'],
    the_type=['jesuita-entrada','partida','chegada','estadia','estadia-x',
                'jesuita-votos-local','jesuita-ordenacao-padre','morte'],
    the_value=place_of_stay,
    column_name='place',
    more_attributes=['jesuita-entrada','nascimento'],
    # filter_by=ids,
    db=tlnb.db,
)

# fillna with "?"
places_of_stay.place = places_of_stay.place.fillna('?')

# remove rows where place == '?'
places_of_stay = places_of_stay[places_of_stay.place != '?']

# this sequence replaces missing dates with the value of the previous date
# followed by ">". This allows to use the information that although the date
# is unkown it has happened after a certain date
# we need the id in a column to group by it (it is currently in the index)
# TODO: maybe an option in entities_with_attribute and compute intervals using
#       post quem and ante quem dates. See https://github.com/time-link/timelink-kleio/issues/1

places_of_stay['id_col'] = places_of_stay.index

# We try to infer the date of stay from the previous date known
# create a copy of the date column to replace the '0' values with NaN
places_of_stay['place.date_inferred'] = places_of_stay['place.date'].replace('0', pd.NA)

# order by id and line to have the missing date values filled with the previous date
places_of_stay = places_of_stay.sort_values(by=['id_col', 'place.line'])
# use ffill to fill the missing values with the previous value
places_of_stay['place.date_inferred'] = places_of_stay.groupby(['id_col'])['place.date_inferred'].fillna(method='ffill')
# create a column to flag the inferred dates
places_of_stay['place.date_is_inferred'] = places_of_stay['place.date_inferred'] != places_of_stay['place.date']
# reset index
places_of_stay.reset_index(inplace=True)
# if data_is_inferred set date_inferred to date_inferrred+">"
places_of_stay.loc[places_of_stay['place.date_is_inferred'], 'place.date_inferred'] = places_of_stay['place.date_inferred'] + '>'
# set missing value to ''
places_of_stay['place.date_inferred'].fillna('', inplace=True)
# restore index
places_of_stay.index = places_of_stay['id_col']

places_of_stay.info()
# show results
show_only=100
cols=['place','place.date_inferred','groupname','name','place.type', 'nascimento', 'nascimento.date', 'place.type','jesuita-entrada', 'jesuita-entrada.date', 'place.date_is_inferred','place.line','place.extra_info']
places_of_stay[places_of_stay.groupname=='n'][cols].sort_values(by=['place','place.date_inferred','name',]).head(show_only)



<class 'pandas.core.frame.DataFrame'>
Index: 6292 entries, deh-abraham-le-royer to simao-rodrigues-ref1
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   id                            6292 non-null   object
 1   name                          6292 non-null   object
 2   groupname                     6292 non-null   object
 3   the_line                      6292 non-null   int64 
 4   the_order                     6292 non-null   int64 
 5   extra_info                    6292 non-null   object
 6   place.attr_id                 6292 non-null   object
 7   place.type                    6292 non-null   object
 8   place                         6292 non-null   object
 9   place.date                    6292 non-null   object
 10  place.line                    6292 non-null   int64 
 11  place.level                   6292 non-null   int64 
 12  place.obs                     6292 non-null   

,place,place.date_inferred,groupname,name,place.type,nascimento,nascimento.date,place.type,jesuita-entrada,jesuita-entrada.date,place.date_is_inferred,place.line,place.extra_info
id_col,,,,,,,,,,,,,
deh-charles-francois-xavier-de-brevedent,Abissínia,16980610,n,Charles-François Xavier de Brévedent,partida,"Genneville, Calvados",16590525,partida,Paris,16740909,False,1347,"{'the_value': {'kleio_element_name': 'valor', ..."
deh-antonio-sedeno,"Acapulco, México",15810329,n,Antonio Sedeño,estadia,"San Clemente, diocese de Cuenca",1535,estadia,Lorette,15590425,False,673,"{'the_date': {'kleio_element_name': 'date', 'k..."
deh-antoine-rene-de-brasle,"Acapulco, México",<1707,n,Antoine-René de Brasle,estadia,"Château-Thierry, diocese de Soissons",16680815,estadia,?,16851006,False,1286,"{'the_value': {'kleio_element_name': 'valor', ..."
deh-antonio-de-magalhaes,Afeganistão,17000000,n,António de Magalhães,estadia,"Cortiços, diocese de Viseu",16770000,estadia,Coimbra,16921019,False,145,"{'entity': {'kleio_element_name': 'entity', 'k..."
deh-pierre-albier,Agen,16540712,n,Pierre Albier,jesuita-votos-local,Limousin,16200000,jesuita-votos-local,?,16440000,False,294,"{'the_date': {'kleio_element_name': 'date', 'k..."
deh-philippe-avril,Agen,16910000,n,Philippe Avril,estadia,Angoulême (paróquia de Notre-Dame de Beaulieu),16540721,estadia,Bordeaux,16700916,False,1416,"{'the_date': {'kleio_element_name': 'date', 'k..."
deh-bento-de-gois,Agra,16021029,n,Bento de Góis,estadia,"Vila Franca do Campo, ilha de S. Miguel, Açores",1563,estadia,?,1584,False,603,"{'entity': {'kleio_element_name': 'entity', 'k..."
deh-bento-de-gois,Agra,16021029,n,Bento de Góis,estadia,"Vila Franca do Campo, ilha de S. Miguel, Açores",1563,estadia,?,1588,False,603,"{'entity': {'kleio_element_name': 'entity', 'k..."
deh-antonio-de-andrade,Agra,16240330,n,António de Andrade,estadia,"Oleiros, diocese de Portalegre",15800000,estadia,Coimbra,15961216,False,1037,"{'the_date': {'kleio_element_name': 'date', 'k..."


### Save to Excel

In [29]:
places_of_stay[cols].sort_values(by=['place','place.date_inferred']).to_excel(f"../inferences/jesuita_entrada_places_of_stay.xlsx", sheet_name='wikidata', index=True)

#### Export to Excel all unlocated places

In [30]:
places_of_stay[ ~(places_of_stay['place.obs'].str.contains('wikidata', na=False))][cols].sort_values(by=['place','place.date_inferred']).to_excel(f"../inferences/jesuita_entrada_places_of_stay_no_wikidata.xlsx", sheet_name='no_wikidata', index=True)

#### Only specific place of entry, unkown places

In [31]:
places_of_stay_specific = places_of_stay.loc[place_of_entry_ids]
places_of_stay_specific[~(places_of_stay_specific['place.obs'].str.contains('wikidata', na=False))][cols].sort_values(by=['place','place.date_inferred'])

,place,place.date_inferred,groupname,name,place.type,nascimento,nascimento.date,place.type,jesuita-entrada,jesuita-entrada.date,place.date_is_inferred,place.line,place.extra_info
id_col,,,,,,,,,,,,,
deh-antonio-de-magalhaes,Afeganistão,17000000,n,António de Magalhães,estadia,"Cortiços, diocese de Viseu",16770000,estadia,Coimbra,16921019,False,145,"{'entity': {'kleio_element_name': 'entity', 'k..."
deh-antonio-de-magalhaes,Afeganistão,17000000,n,António de Magalhães,estadia,"Cortiços, diocese de Viseu",16770000,estadia,Coimbra,16921019,False,145,"{'entity': {'kleio_element_name': 'entity', 'k..."
deh-antonio-de-andrade,Agra,16240330,n,António de Andrade,estadia,"Oleiros, diocese de Portalegre",15800000,estadia,Coimbra,15961216,False,1037,"{'the_date': {'kleio_element_name': 'date', 'k..."
deh-pedro-martins,Alcácer-Quibir,15780804,n,Pedro Martins,estadia,Coimbra,15420000,estadia,Coimbra,15560525,False,792,"{'entity': {'kleio_element_name': 'entity', 'k..."
deh-pedro-martins,Alcácer-Quibir,15780804,n,Pedro Martins,estadia,Coimbra,15420000,estadia,Coimbra,15560525,False,792,"{'entity': {'kleio_element_name': 'entity', 'k..."
deh-francisco-pacheco,"Arima, Japão",,n,Francisco Pacheco,estadia,"Ponte de Lima, diocese de Braga",15680000,estadia,Coimbra,15860000,True,66,"{'entity': {'kleio_element_name': 'entity', 'k..."
deh-sebastiao-vieira,Avignon,16261225,n,Sebastião Vieira,estadia,"Castro Daire, diocese de Lamego",1574,estadia,Coimbra,15910203,False,1353,"{'entity': {'kleio_element_name': 'entity', 'k..."
deh-sebastiao-vieira,Avignon,16261225,n,Sebastião Vieira,estadia,"Castro Daire, diocese de Lamego",1574,estadia,Coimbra,15910203,False,1353,"{'entity': {'kleio_element_name': 'entity', 'k..."
deh-belchior-nunes-barreto,Baçaim,1553,n,Belchior Nunes Barreto,estadia-x,Porto,15190000,estadia-x,Coimbra,15430311,False,373,"{'the_type': {'kleio_element_name': 'tipo', 'k..."


In [32]:
places_of_stay_specific[~(places_of_stay_specific['place.obs'].str.contains('wikidata', na=False))][cols].sort_values(by=['place','place.date_inferred']).to_excel(f"../inferences/jesuita_entrada_{place_of_entry}_places_of_stay_no_wikidata.xlsx", sheet_name='no_wikidata', index=True)

### Frequency of places / Frequência dos lugares

In [33]:
places_of_stay.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6292 entries, deh-abraham-le-royer to simao-rodrigues-ref1
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   id                            6292 non-null   object
 1   name                          6292 non-null   object
 2   groupname                     6292 non-null   object
 3   the_line                      6292 non-null   int64 
 4   the_order                     6292 non-null   int64 
 5   extra_info                    6292 non-null   object
 6   place.attr_id                 6292 non-null   object
 7   place.type                    6292 non-null   object
 8   place                         6292 non-null   object
 9   place.date                    6292 non-null   object
 10  place.line                    6292 non-null   int64 
 11  place.level                   6292 non-null   int64 
 12  place.obs                     6292 non-null   

In [34]:
col = 'place' # subtotal by this column

# Use pandas groupby and specify unique value count for id
df_totals = places_of_stay.groupby(col).agg({'id':'nunique',
                                                'place.date_inferred':'min',
                                                'place.date':'max'})
df_totals.info()
df_totals.sort_values('id',ascending= False).head(130)

<class 'pandas.core.frame.DataFrame'>
Index: 1099 entries, Abissínia to Índia Oriental
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   1099 non-null   int64 
 1   place.date_inferred  1099 non-null   object
 2   place.date           1099 non-null   object
dtypes: int64(1), object(2)
memory usage: 34.3+ KB


,id,place.date_inferred,place.date
place,,,
Macau,449,,>1652
Pequim,250,,>171410115
Goa,209,,>1713
Cantão,197,,>16590123
China,180,,18050114
Lisboa,130,,17810300
Roma,110,,>16851112
Coimbra,77,,17530000
Paris,65,,17780000


## Priesthood / Ordens sacras

Those that received the sacrament of priesthood in a given place

In [22]:
import pyuca  # to sort accented characters properly

import pandas as pd
from timelink.pandas import entities_with_attribute
# show 500 rows
pd.set_option('display.max_rows', 550)

jesuita_ordenacao_padre = entities_with_attribute(
    entity_type='person',
    show_elements=['name','groupname'],
    the_type='jesuita-ordenacao-padre',
    the_value='Coimbra',
    more_attributes=['dehergne','jesuita-entrada','jesuita-votos-local','nascimento', ],
    db=tlnb.db,
)
jesuita_ordenacao_padre.info()

coll = pyuca.Collator()

# we use the Collator class to sort the names and countries properly
# since Python does not sort accented characters properly
# we create two new columns with the sort keys

# first we fill the NaN values with empty strings
jesuita_ordenacao_padre['jesuita-votos-local'] = jesuita_ordenacao_padre['jesuita-votos-local'].fillna('?')
jesuita_ordenacao_padre['jesuita-votos-local'] = jesuita_ordenacao_padre['jesuita-votos-local'].fillna('?')

# now we add two columns with sort keys
jesuita_ordenacao_padre['jesuita-votos-loc-sort'] = jesuita_ordenacao_padre['jesuita-votos-local'].apply(coll.sort_key)
jesuita_ordenacao_padre['name_sort'] = jesuita_ordenacao_padre['name'].apply(coll.sort_key)
jesuita_ordenacao_padre

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, deh-pedro-ramon to deh-manuel-de-siqueira-tcheng
Data columns (total 38 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   name                                  3 non-null      object
 1   groupname                             3 non-null      object
 2   jesuita-ordenacao-padre.attr_id       3 non-null      object
 3   jesuita-ordenacao-padre.type          3 non-null      object
 4   jesuita-ordenacao-padre               3 non-null      object
 5   jesuita-ordenacao-padre.date          3 non-null      object
 6   jesuita-ordenacao-padre.line          3 non-null      int64 
 7   jesuita-ordenacao-padre.level         3 non-null      int64 
 8   jesuita-ordenacao-padre.obs           3 non-null      object
 9   jesuita-ordenacao-padre.extra_info    3 non-null      object
 10  jesuita-ordenacao-padre.comment       3 non-null      object
 11 

,name,groupname,jesuita-ordenacao-padre.attr_id,jesuita-ordenacao-padre.type,jesuita-ordenacao-padre,jesuita-ordenacao-padre.date,jesuita-ordenacao-padre.line,jesuita-ordenacao-padre.level,jesuita-ordenacao-padre.obs,jesuita-ordenacao-padre.extra_info,jesuita-ordenacao-padre.comment,jesuita-ordenacao-padre.date.comment,dehergne,dehergne.date,dehergne.obs,dehergne.extra_info,dehergne.comment,jesuita-entrada,jesuita-entrada.date,jesuita-entrada.obs,jesuita-entrada.extra_info,jesuita-entrada.original,jesuita-entrada.comment,jesuita-entrada.date.comment,jesuita-votos-local,jesuita-votos-local.date,jesuita-votos-local.obs,jesuita-votos-local.extra_info,jesuita-votos-local.date.comment,jesuita-votos-local.original,jesuita-votos-local.comment,nascimento,nascimento.date,nascimento.obs,nascimento.extra_info,nascimento.comment,nascimento.date.comment,nascimento.original,jesuita-votos-loc-sort,name_sort
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
deh-pedro-ramon,Pedro Ramón,n,deh-pedro-ramon-att22-194,jesuita-ordenacao-padre,Coimbra,1573,42,3,,"{'the_date': {'kleio_element_name': 'date', 'k...",@wikidata:Q45412,None,670,0,"Rámon, Pedro (espagn.) P. 670 N. « Castellano ...","{'the_date': {'kleio_element_name': 'date', 'k...",None,Alcalá,1571,,"{'the_date': {'kleio_element_name': 'date', 'k...",de Henares,@wikidata:Q46940,None,Amakusa,15910801,,"{'the_date': {'kleio_element_name': 'date', 'k...",e 15910803,Amakusae,@wikidata:Q452622,Saragoça,1550,,"{'the_date': {'kleio_element_name': 'date', 'k...",@wikidata:Q31926034,None,None,"(7239, 7594, 7239, 7525, 7861, 7793, 7239, 0, ...","(7692, 7338, 7311, 7731, 7645, 521, 7731, 7239..."
deh-duarte-de-sande,Duarte de Sande,n,deh-duarte-de-sande-att220-188,jesuita-ordenacao-padre,Coimbra,1577,293,3,,"{'the_date': {'kleio_element_name': 'date', 'k...",@wikidata:Q45412,None,741,0,"""""""\n Sande, Duarte (Edoardo) de (p...","{'the_date': {'kleio_element_name': 'date', 'k...",@archive:276,Lisboa,15620600,,"{'the_date': {'kleio_element_name': 'date', 'k...",None,@wikidata:Q597,None,Goa,15840115,,"{'the_date': {'kleio_element_name': 'date', 'k...",None,None,@wikidata:Q1171,Guimarães,15471025,,"{'the_date': {'kleio_element_name': 'date', 'k...",@wikidata:Q170903,antes de,None,"(7412, 7645, 7239, 0, 32, 32, 32, 0, 8, 2, 2, 0)","(7311, 7861, 7239, 7731, 7829, 7338, 521, 7311..."
deh-manuel-de-siqueira-tcheng,Manuel de Siqueira Tcheng,n,deh-manuel-de-siqueira-tcheng-att1183-188,jesuita-ordenacao-padre,Coimbra,1664,1631,3,,"{'the_date': {'kleio_element_name': 'date', 'k...",@wikidata:Q45412,morre antes dos seus grandes votos,790,0,"""""""\n Siqueira Tcheng, Manuel de (m...","{'the_date': {'kleio_element_name': 'date', 'k...",@archive:291,Roma,16511017,"segundo a entrada de Rhodes (677), este baptiz...","{'the_date': {'kleio_element_name': 'date', 'k...",None,@wikidata:Q220,vem com o P. Alexandre de Rhodes,?,NaN,NaN,NaN,NaN,NaN,NaN,Macau,0,"25 maio 1653 é a data que aparece, mas não pod...","{'the_date': {'kleio_element_name': 'date', 'k...",@wikidata:Q14773,None,«Sinensis natus Macai»,"(614, 0, 32, 0, 2, 0)","(7594, 7239, 7609, 7861, 7338, 7543, 521, 7311..."


## Vows / Votos

In [23]:
import pyuca  # to sort accented characters properly

import pandas as pd
from timelink.pandas import entities_with_attribute
# show 500 rows
pd.set_option('display.max_rows', 550)

jesuita_entrada_full = entities_with_attribute(
    entity_type='person',
    show_elements=['name','groupname'],
    the_type='jesuita-votos',
    more_attributes=['jesuita-votos-local','nascimento', 'jesuita-entrada'],
    db=tlnb.db,
)
jesuita_entrada_full.info()

coll = pyuca.Collator()

# we use the Collator class to sort the names and countries properly
# since Python does not sort accented characters properly
# we create two new columns with the sort keys

# first we fill the NaN values with empty strings
jesuita_entrada_full['jesuita-votos-local'] = jesuita_entrada_full['jesuita-votos-local'].fillna('?')
jesuita_entrada_full['jesuita-votos-local'] = jesuita_entrada_full['jesuita-votos-local'].fillna('?')

# now we add two columns with sort keys
jesuita_entrada_full['jesuita-votos-loc-sort'] = jesuita_entrada_full['jesuita-votos-local'].apply(coll.sort_key)
jesuita_entrada_full['name_sort'] = jesuita_entrada_full['name'].apply(coll.sort_key)


<class 'pandas.core.frame.DataFrame'>
Index: 628 entries, deh-abraham-le-royer to joao-cardoso
Data columns (total 34 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   name                              628 non-null    object
 1   groupname                         628 non-null    object
 2   jesuita-votos.attr_id             628 non-null    object
 3   jesuita-votos.type                628 non-null    object
 4   jesuita-votos                     628 non-null    object
 5   jesuita-votos.date                628 non-null    object
 6   jesuita-votos.line                628 non-null    int64 
 7   jesuita-votos.level               628 non-null    int64 
 8   jesuita-votos.obs                 628 non-null    object
 9   jesuita-votos.extra_info          628 non-null    object
 10  jesuita-votos.date.comment        31 non-null     object
 11  jesuita-votos.date.original       2 non-null      object
 12 

In [24]:
# filter dataframe with rows with index in place_of_entry_ids


In [25]:
from timelink.kleio.utilities import convert_timelink_date

df = jesuita_entrada_full[jesuita_entrada_full.index.isin(place_of_entry_ids)].copy()
# calculate age at date "jesuita-votos.date" considering "nascimento.date" as date of birth
# and using the function convert_timelink_date to convert the Timelink date to a Python datetime
df['nascimento.date_2'] = df['nascimento.date'].fillna('0').apply(convert_timelink_date)
df['jesuita-votos.date_2'] = df['jesuita-votos.date'].fillna('0').apply(convert_timelink_date)
# calculate diference in years between the two dates
df['age_at_votos'] = df.apply(lambda row: calc_age_at(row['nascimento.date_2'], row['jesuita-votos.date_2']), axis=1)
df['age_at_votos'] = df['age_at_votos'].fillna(0).astype(int)
df[['nascimento.date','jesuita-votos.date','jesuita-votos','age_at_votos']].head(50)


,nascimento.date,jesuita-votos.date,jesuita-votos,age_at_votos
id,,,,
deh-andre-carneiro,1650,16821208,4V,32
deh-andre-ferrao,1625,16590116,4V,33
deh-andre-palmeiro,15690000,16030727,4V,34
deh-antao-dantas,16741102,17110815,4V,36
deh-antonio-da-silva,16540113,16870815,4V,33
deh-antonio-de-andrade,15800000,16121014,4V,32
deh-antonio-de-gouvea,1592,16460714,4V,54
deh-antonio-de-magalhaes,16770000,17110202,4V,33
deh-antonio-de-melo,16810803,17191203,4V,38


## Voyages


In [ ]:
import pandas as pd
from timelink.pandas import entities_with_attribute
# show 500 rows
pd.set_option('display.max_rows', 550)

voyage_of_interest = 25

voyage = entities_with_attribute(
    entity_type='person',
    show_elements=['name','groupname'],
    the_type='wicky-viagem',
    the_value=str(voyage_of_interest),
    more_attributes=['embarque','chegada','partida',],
    db=tlnb.db,
)
voyage

,name,groupname,wicky-viagem.type,wicky-viagem,wicky-viagem.date,wicky-viagem.line,wicky-viagem.level,wicky-viagem.obs,embarque,embarque.date,embarque.obs,chegada,chegada.date,chegada.obs,partida,partida.date,partida.obs
id,,,,,,,,,,,,,,,,,
deh-duarte-de-sande,Duarte de Sande,n,wicky-viagem,25,15780324,290,4,,S. Luís,15780324,,NaN,NaN,NaN,Macau,15850501,
deh-francesco-pasio,Francesco Pasio,n,wicky-viagem,25,15780324,372,4,,S. Gregório,15780324,,Macau,16120400,,Macau,16120322,
deh-giovanni-gerardino,Giovanni Gerardino,n,wicky-viagem,25,15780324,384,4,,Bom Jesus,15780324,,NaN,NaN,NaN,NaN,NaN,NaN
deh-matteo-ricci,Matteo Ricci,n,wicky-viagem,25,15780324,571,4,,S. Luís,15780324,,Goa,15780913,,Pequim,16000519,
deh-matteo-ricci,Matteo Ricci,n,wicky-viagem,25,15780324,571,4,,S. Luís,15780324,,Macau,15820807,,Pequim,16000519,
deh-matteo-ricci,Matteo Ricci,n,wicky-viagem,25,15780324,571,4,,S. Luís,15780324,,Pequim,16010124,,Pequim,16000519,
deh-michele-ruggiere,Michele (Pompilio) Ruggiere,n,wicky-viagem,25,15780324,1956,4,,S. Luís,15780324,,Macau,15790720,,Roma,15881100,


## Visualizar registos / View records





### Visualizar uma pessoa / View a person


#### Atributos de uma pessoa numa tabela, uma linha por attributo

---

#### Person attributes in a dataframe, one line per attribute

In [ ]:
import pandas as pd
from timelink.pandas import group_attributes as person_attributes

pd.set_option('display.max_rows',1000)

id = 'deh-matteo-ricci'
pdf = person_attributes([id],db=tlnb.db)  # note id in a list
pdf[['the_date','the_type','the_value','attr_obs']].sort_values(['the_date','the_type'])

,the_date,the_type,the_value,attr_obs
id,,,,
deh-matteo-ricci,0,dehergne,684,"Ricci, Matteo (ital.) P. 684 Li Ma-teou, Si-T'..."
deh-matteo-ricci,0,dehergne,684,"P. 25 juill. 1580, à Cochin, où il célèbre sa ..."
deh-matteo-ricci,0,dehergne,684,"D'Elu III, 239-243. Biographie: en chinois: BN..."
deh-matteo-ricci,0,dehergne,684,"Tacchi-Venturi, Opère storiche del P. M. R., M..."
deh-matteo-ricci,0,dehergne,684,id.. L'apport scientifique du P. M. R à la Chi...
deh-matteo-ricci,0,dehergne,684,"Bettray, Johannes, S.V.D., Die akkomodationmet..."
deh-matteo-ricci,0,dehergne,684,"Sur l'art chinois: Sepp Schûller, P. M. Ricci ..."
deh-matteo-ricci,0,dehergne,684,"— Délia Entrata delta Comp. nella Cina, avec a..."
deh-matteo-ricci,0,dehergne,684,"pécimen d'écriture, nom chinois et romanisatio..."


In [ ]:
p = tlnb.db.get_person(id)
print(p.to_kleio())

n$Matteo Ricci/m/id=deh-matteo-ricci
  <rel$parentesco/pai/Giovanni Battista Ricci/deh-matteo-ricci-pai/
  <rel$parentesco/mae/Giovanna Angiolelli/deh-matteo-ricci-mae/
  rel$function-in-act/n/lista/dehergne-notices-r//obs=line: 556
  ls$nacionalidade/Itália/
  ls$jesuita-estatuto/Padre/
  ls$nome-chines/Li Ma-teou/
  ls$nome-chines/Si-T'ai/
  ls$tarefa/Calendário cristão na China#introduz?/
  ls$dehergne/684/obs=Ricci, Matteo (ital.) P. 684 Li Ma-teou, Si-T'al (Pf.., et non Ki-Jen, RBS 218)., en portugais Richo (DI. XI, 155, n. 19). N. 6 oct. 1552, Macerata, dans la Marche d'Ancône. E. noviciat St André du Quirinal, Rome, 16 août 1571 (Rom. 171A, 16: il signe: Matteo Riccio). Emb. sur le S. Luis, 24 mars 1578 (W 200)., arr. Goa, 13 sept., et Macao, 7 août 1582., août 1583, Canton., 10 sept. 1583, Shiuhing (Tchao-k'ing fou), où il fonde une résidence., oct. 1589, résidence à Shiuchow (Chao-tcheou)., du 21 mai 1595 au 15 juin, Nankin., 28 juin 1596 au 25 juin 1598, Nan-tch'ang fou., 5 j

#### Atributos de uma pessoa numa tabela, attributos em colunas

---

#### Person attributes in a dataframe, attributes in columns

In [ ]:
# Get list of people with with a certain value in a specific attribute

id = 'deh-andre-palmeiro'  # João Pedro Ribeiro
# id = '234295'  # Alexandre Metelo de Sousa
df = entities_with_attribute(
                    entity_type='person',
                    the_type='jesuita-entrada',  # we need a base attribute
                    show_elements=['name'],
                    more_attributes=['jesuita-votos','jesuita-votos-local'],
                    db=tlnb.db,
                    filter_by=[id],
                    sql_echo=False)
view_cols = ['name','jesuita-entrada','jesuita-entrada.date','jesuita-votos','jesuita-votos-local']
df.info()
df[view_cols].sort_values('jesuita-entrada.date')

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, deh-andre-palmeiro to deh-andre-palmeiro
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   name                      1 non-null      object
 1   jesuita-entrada.type      1 non-null      object
 2   jesuita-entrada           1 non-null      object
 3   jesuita-entrada.date      1 non-null      object
 4   jesuita-entrada.line      1 non-null      int64 
 5   jesuita-entrada.level     1 non-null      int64 
 6   jesuita-entrada.obs       1 non-null      object
 7   jesuita-votos             1 non-null      object
 8   jesuita-votos.date        1 non-null      object
 9   jesuita-votos.obs         1 non-null      object
 10  jesuita-votos-local       1 non-null      object
 11  jesuita-votos-local.date  1 non-null      object
 12  jesuita-votos-local.obs   1 non-null      object
dtypes: int64(2), object(11)
memory usage: 220.0+ bytes


,name,jesuita-entrada,jesuita-entrada.date,jesuita-votos,jesuita-votos-local
id,,,,,
deh-andre-palmeiro,André Palmeiro,Coimbra,15840114,4V,Coimbra


#### Examinar potenciais duplicados

---

#### Examine potentital duplicates

In [9]:
from timelink.pandas import display_group_attributes
pd.set_option('display.max_rows',250)

no_show=['código-de-referência','data-do-registo','url','faculdade.ano','naturalidade.ano',
         'matricula-faculdade.ano','nome-apelido','nome-primeiro','nome-geografico.ano',
         'grau.ano','matricula-outra.ano','nome-geografico','instituta.ano']

dup_ids = ['deh-joao-barradas-ref1','deh-lazzaro-cattaneo-ref1','deh-matteo-ricci']  # Alexandre Metelo de

display_group_attributes(dup_ids,
                             header_attributes=['jesuita-entrada','nacionalidade'],
                             exclude_attributes=no_show,
                             sort_attributes=['the_date','the_type','the_value'],
                             cmap_name='Pastel1',
                             db=tlnb.db,)


,jesuita-entrada.type,jesuita-entrada,jesuita-entrada.date,jesuita-entrada.obs,jesuita-entrada.original,jesuita-entrada.original.comment,nacionalidade,nacionalidade.date,nacionalidade.obs,id
0,jesuita-entrada,Roma,15710816,,(Noviciado de Sant'Andrea al Quirinale),@wikidata:Q220,Itália,0,,deh-matteo-ricci
1,nan,nan,nan,nan,nan,nan,nan,nan,nan,deh-joao-barradas-ref1
2,nan,nan,nan,nan,nan,nan,nan,nan,nan,deh-lazzaro-cattaneo-ref1


,the_type,the_value,the_date,attr_obs,id
0,dehergne,684,0,"Ricci, Matteo (ital.) P. 684 Li Ma-teou, Si-T'al (Pf.., et non Ki-Jen, RBS 218)., en portugais Richo (DI. XI, 155, n. 19). N. 6 oct. 1552, Macerata, dans la Marche d'Ancône. E. noviciat St André du Quirinal, Rome, 16 août 1571 (Rom. 171A, 16: il signe: Matteo Riccio). Emb. sur le S. Luis, 24 mars 1578 (W 200)., arr. Goa, 13 sept., et Macao, 7 août 1582., août 1583, Canton., 10 sept. 1583, Shiuhing (Tchao-k'ing fou), où il fonde une résidence., oct. 1589, résidence à Shiuchow (Chao-tcheou)., du 21 mai 1595 au 15 juin, Nankin., 28 juin 1596 au 25 juin 1598, Nan-tch'ang fou., 5 juill. 1599, Nankin., 19 mai 1600, départ pour Pékin où il arr. le 24 janv. 1601, et où il fonde la mission du Hopei, 1601., Supérieur de la mission de Chine, d'août 1597 à sa mort., 1609, congrégation mariale. Pékin., calendrier chrétien en Chine (Pf. 117).",deh-matteo-ricci
1,dehergne,684,0,"P. 25 juill. 1580, à Cochin, où il célèbre sa première messe le 26 juill. (D'Elia III, 22). V. Nan-tch'ang 1er janv. 1596 (D'Elia I, 374 et II, 4) pr. « Nanciani » ... Riccius (Lus. 2, 128). M. 11 mai 1610, Pékin (JS 113, 120-146., Planchet p. 222). Fils de Giovanni Battista Ricci, pharmacien, et de Giovanna Angiolelli, famille noble, marquis de Castelvecchio (D'Elia I, cii et II, 550). HS 43, 58 le fait mourir d'abord en 1609, puis le 11 mai 1610. Portrait: Pf. 31., D'Elia I, p. vi., ARSI Rom. 185, f° 97-98., BN: Estampes, Ne 100., en couleurs dans Bortone, P. Fernando,- Gesuiti alla corte di Pechino, 1969, pl. II p. 1., cf. Hamy. Pf. 22., Koch., NCE (F. A. Rouleau)., EC (Cel. Testore)., EB (J.S. Sebes)., CE (J. Brucker)., El (Tacchi Venturi). Bibliographie: SiRErr IV, p. 525-528., XII, p. 275-283., XIV. 1. p. 778.,",deh-matteo-ricci
2,dehergne,684,0,"D'Elu III, 239-243. Biographie: en chinois: BN: Courant 9 (anonyme), 1014-1016 I, par Aleni, 1321 (par Pantoja, de Ursis)., et 6832 I, poème sur Rica. BernardE-MaÎtre], H., Le P. Matthieu Ricci et la société chinoise de son temps (1552-1610), Tientsin, 1937, 2 vol.., trad. chinoise par Wang Tch'ang-Tche, J., 1943., du même, Iles Philippines, 213-218 (sur la pensée de Ricci)., Cronin, Vincent, The Wise Man front the West, Londres, 1955., Dunne, G. H., Génération of Giants, 1962, p. 23-108., trad. franç. Chinois avec les Chinois, te P. Ricci et ses compagnons, Paris, 1964., Bortone, Fernando, P. Matteo Ricci, SJ.: Il « Saggio d'Occidente », 2' éd., 1965., trad. franç. en préparation. Le ms de Pires, Francisco, montre qu'il a failli être envoyé au Japon.Son oeuvre: Nocentini, Lod., --primo Sinologo, P.M.R., Florence, 1882.,",deh-matteo-ricci
3,dehergne,684,0,"Tacchi-Venturi, Opère storiche del P. M. R., Macerata, 1911-1912, 2 vol.., D'Elia, P. M., Fonti Ricciane, Rome, 1942-1949, 3 vol.., du même,. Grandi Missionari, t. I (Rome, 1939), 127-176., du même, Il domma cattolico integralmente présentai da M. R ai letterati delta Cina, Civ. Catt., 1935, II, 35-53., cf Gregorianum 21 (1940), 482-526., du même, Galiteo in Cina, Anatecta Gregor. il, Sect. A n° 1., Son oeuvre: Nocentini, Lod., -- primo Sinologo, P.M.R., Florence, 1882., Tacchi-Venturi, Opère storiche del P. M. R., Macerata, 1911-1912, 2 vol.., D'Elia, P. M., Fonti Ricciane, Rome, 1942-1949, 3 vol.., du même,- Grandi Missionari, t. I (Rome, 1939), 127-176., du même, Il domma cattolico integralmente présentai da M. R ai letterati delta Cina, Civ. Catt., 1935, II, 35-53., cf Gregorianum 21 (1940), 482-526., du même, Galiteo in Cina, Anatecta Gregor. il, Sect. A n° 1.,",deh-matteo-ricci
4,dehergne,684,0,"id.. L'apport scientifique du P. M. R à la Chine, Tientsin, 1935, dont trad. anglaise par Chalmers Werner, Edw., Pékin, 1935., id., L'attitude du P. M. R. en face des coutumes et rites chinois, RSRel. 28 (1938), 31-47., id., Les étapes de la cartographie scientifique pour ta Chine et tes pays voisins, MS I (1936), 428-477., Brou, A., Les tâtonnements du P. M. R., RHM 15 (1938), 228-244., Szczesniak, B., The Seve

## Other



###  Remover colunas sem valores

---



###  Remove empty columns

In [ ]:
jesuita_entrada_embarque.dropna(how='all', axis=1, inplace=True)
jesuita_entrada_embarque.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, deh-pedro-de-alcacova to deh-inacio-pires
Data columns (total 36 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   name                           59 non-null     object
 1   groupname                      59 non-null     object
 2   extra_info                     59 non-null     object
 3   jesuita-entrada.type           59 non-null     object
 4   jesuita-entrada                59 non-null     object
 5   jesuita-entrada.date           59 non-null     object
 6   jesuita-entrada.line           59 non-null     int64 
 7   jesuita-entrada.level          59 non-null     int64 
 8   jesuita-entrada.obs            59 non-null     object
 9   dehergne                       59 non-null     object
 10  dehergne.date                  59 non-null     object
 11  dehergne.obs                   59 non-null     object
 12  nascimento                     59 non

In [ ]:
jesuita_entrada_embarque.head(5)

,name,groupname,extra_info,jesuita-entrada.type,jesuita-entrada,jesuita-entrada.date,jesuita-entrada.line,jesuita-entrada.level,jesuita-entrada.obs,dehergne,...,nascimento.date.formated,nascimento.date.year,jesuita-entrada.date.formated,jesuita-entrada.date.year,embarque.date.formated,embarque.date.year,morte.date.formated,morte.date.year,ano_entrada,periodo_entrada
id,,,,,,,,,,,,,,,,,,,,,
deh-pedro-de-alcacova,Pedro de Alcáçova,n,{},jesuita-entrada,Coimbra,15420000,296,4,saiu a primeira vez,15,...,1523,1523,1542,1542,,,1579,1579,1542,1540
deh-belchior-nunes-barreto,Belchior Nunes Barreto,n,{},jesuita-entrada,Coimbra,15430311,380,4,,589,...,1519,1519,1543-03-11,1543,1551-03-10,1551,1571-08-10,1571,1543,1540
deh-belchior-miguel-carneiro-leitao,Belchior Miguel Carneiro Leitão,n,{},jesuita-entrada,Coimbra,15430425,487,4,,143,...,1519,1519,1543-04-25,1543,1555-04-01,1555,1583-08-19,1583,1543,1540
deh-francisco-perez,Francisco Pérez,n,{},jesuita-entrada,Coimbra,15440125,867,4,,628,...,1514,1514,1544-01-25,1544,1546-04-08,1546,1583-02-22,1583,1544,1540
deh-tiburcio-de-quadros,Tibúrcio de Quadros,n,{},jesuita-entrada,Coimbra,15440418,14,4,,666,...,1528,1528,1544-04-18,1544,1555-04-01,1555,1572-11-21,1572,1544,1540



### Contagens

---

### Counting



###  Contagem de atributos a partir de uma tabela em memória

---

###  Count attributes from an existing dataframe



In [ ]:
jesuita_entrada_embarque.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, deh-pedro-de-alcacova to deh-inacio-pires
Data columns (total 36 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   name                           59 non-null     object
 1   groupname                      59 non-null     object
 2   extra_info                     59 non-null     object
 3   jesuita-entrada.type           59 non-null     object
 4   jesuita-entrada                59 non-null     object
 5   jesuita-entrada.date           59 non-null     object
 6   jesuita-entrada.line           59 non-null     int64 
 7   jesuita-entrada.level          59 non-null     int64 
 8   jesuita-entrada.obs            59 non-null     object
 9   dehergne                       59 non-null     object
 10  dehergne.date                  59 non-null     object
 11  dehergne.obs                   59 non-null     object
 12  nascimento                     59 non

In [ ]:
# create a column with the index values which are the id numbers
jesuita_entrada_embarque['id'] = jesuita_entrada_embarque.index.values

col = 'jesuita-entrada' # subotal by this column

# Use pandas groupby and specify unique value count for id
df_totals = jesuita_entrada_embarque.groupby(col).agg({'id':'nunique',
                                                  'jesuita-entrada.date':'min',
                                                  'jesuita-entrada.date':'max'})

df_totals.sort_values('id',ascending= False).head(30)

,id,jesuita-entrada.date
jesuita-entrada,,
Coimbra,59,17420427



### Contagens na base de dados

Quando o atributo tem muitos valores e não é necessário
ter todas as pessoas em memória: contagem feita na base de dados

---

### Counting directly in the database
When there are many values and it is not
necessary to have all the people in memory:
count directly in the database.




In [ ]:
from timelink.pandas import attribute_values

df_totals = attribute_values('jesuita-entrada',db=tlnb.db)


In [ ]:
df_totals.head(10)


,count,date_min,date_max
value,,,
?,484,15440000,17880828
Coimbra,60,15420000,17420427
Lisboa,45,15460000,17530612
Paris,41,16300108,17590310
Roma,40,15480311,17560709
Goa,33,15480000,17360000
Évora,26,15660714,17460405
Nancy,15,16270929,17510827
Landsberg,14,16230729,17571009


#### Filtrar por datas

---

#### Filter by dates



In [ ]:
df_totals = attribute_values('grau',dates_between=('1535','1600'),db=tlnb.db)

In [ ]:
df_totals.head(10)

,value,count,date_min,date_max
value,NaN,NaN,NaN,NaN
